In [57]:
##### DOWNLOAD DATASET #####
from IPython.display import clear_output
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

In [58]:
########### SPARK CONTEXT #####################
import pandas as pd

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext
###############################################

In [59]:
############## BASKET CREATION ################
#TODO do it using SPARK directly -> cars = spark.read.csv('cars.csv', header=True, sep=";")
actors = pd.read_csv("letterbox/actors.csv")
baskets = actors.groupby("id")["name"].apply(list)

print("number of baskets: " + str(len(baskets)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)
baskets_RDD = sc.parallelize(baskets).cache()
##############################################

number of baskets: 603163
biggest basket: 451
id
1000001    [Margot Robbie, Lewis Easter, Onyemachi Ejimof...
1000002    [Rosie Peralta, Anna Elisabeth Rihlmann, Andre...
1000003    [Randall Archer, Boon Pin Koh, Efka Kvaracieju...
1000004    [Leonard Termo, Greg Bronson, Michael Arturo, ...
1000005    [Lena Georgas, Jeff Hephner, Elyes Gabel, Broo...
                                 ...                        
1896377    [Marat Basharov, Sergey Rost, Gleb Kalyuzhny, ...
1896382                                   [Marine Petrosyan]
1896387                                        [Rebecca Jim]
1896391                                       [Ko Shibasaki]
1896393                                    [Atsuhiro Inukai]
Name: name, Length: 603163, dtype: object


In [65]:
baskets_RDD.getNumPartitions()

2

In [90]:
## APRIORI
def apriori(chunk, s):

  n_bsk = 0 # number of baskets in chunk used to calculate support
  i = 0
  trsl = dict()

  def translate(e, dic):
    try:
      return dic[e]
    except KeyError:
      dic[e] = len(dic)+1
      return dic[e]


  for bsk in chunk:
    n_bsk += 1;
    translated = [translate(x,trsl) for x in bsk]



  def first_pass():
    pass

  def second_pass():
    pass


  yield (len(trsl),1)

In [93]:

def first_red(key,value):
  yield key
def second_map(candidate, chunk):
  yield (1,1)
def second_red(key,value):
  yield key

chunk = 1; #TMP to remove
s = 10; #support threshold

baskets_RDD.mapPartitions(lambda x: apriori(x,s)).reduceByKey(lambda a,b: a).collect()


[(1039348, 1), (788811, 1)]